https://www.accesscanberra.act.gov.au/s/contstruction-professionals

In [ ]:
import selenium
import time
from selenium import webdriver

from bs4 import BeautifulSoup
import pandas as pd

## Ignoring Warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
driver = webdriver.Chrome('/Users/rohit_manral/Downloads/chromedriver')

# Getting the access of `ACT Builder's` official website

In [ ]:
driver.get('https://www.accesscanberra.act.gov.au/s/public-registers/construction-licences?registerid=m0I4a00000000CHEAY')
time.sleep(2)

#### Finding number of pages of the data 

In [ ]:
pages=driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/p').text
#pages

numbers = []
for word in pages.split():
    if word.isdigit():
        numbers.append(int(word))
print(numbers[1]) #no. of pages

## First Page  
### Main program to get `Headers` & `all the data rows` of just the `first page`

In [ ]:
page_source = driver.page_source
soup =  BeautifulSoup(page_source, 'lxml')
            
headers = [i.text for i in soup.findAll('th')]
headers = headers[:-1]
print(headers)

In [ ]:
xpaths=[]
for j in range(1,11): #creating xpaths: just change span[i] to click different links
    xpaths.append('/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[5]/div/div/table/tbody/tr['+str(j) + ']/td[6]/lightning-button/button')
#print(xpaths) 

## All pages now  
### Main program to get `all the data rows` of all the pages `till last page`

In [ ]:
licensee_detail=[]

In [ ]:
for i in range(1, numbers[1]+1):
    
    page_source = driver.page_source
    soup =  BeautifulSoup(page_source, 'lxml')       
    
    data = [j.text for j in soup.findAll('td')]
    all_names= data[::6]
    
    
    for k in range(1,11): 
        
        driver.find_element_by_xpath(xpaths[k-1]).click()
        #time.sleep(1)
    
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
            
        det= [l.text for l in soup.findAll('td')]
        det=det[1:]
        #det

        ab=[]
        ab=det[::2]

        ab.insert(0,all_names[k-1])
        #ab

        licensee_detail.append(ab)
    
        driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div/lightning-button/button').click()
        
        
    # click to next page
    driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/div/div/div/div/div/div[3]/div/div/div[2]/c-cxs-public-registers-construction/div/div[6]/div[4]/button').click()
    #time.sleep(1)

In [ ]:
licensee_detail[0]

In [ ]:
licensee_detail[-1]

In [ ]:
len(licensee_detail)

In [ ]:
df_builder = pd.DataFrame(licensee_detail, columns = ['Name of Individual or Company', 'License Number','Expiry Date','Occupation','Class','Class Condition / Endorsement'])
df_builder

In [ ]:
df_builder.shape

In [ ]:
df_builder.to_csv('ACT_Builders.csv')